# Sunburst

In [ ]:
<svg id="viz"></svg>


In [ ]:
let dtreeLoader = interactive.configureRequire({
    paths: {
        d3: "https://d3js.org/d3.v6.min"
    }
});

dtreeLoader(["d3"], function (d3) {
    console.log("Loading d3")
})

Loading d3

In [ ]:
define('getfiledata', ['d3'], function(d3) {

    var url = 'https://static.observableusercontent.com/files/4b8bc441afab87356f7b5cc5aef3130f4ca634aaae3a46ba7c0f7950b152bc9cdd9bbddae444d694f3e3b4d43587419a17eb0bd5fbd340ce6d6c7b31907bfc7b?response-content-disposition=attachment%3Bfilename*%3DUTF-8%27%27visit-sequences%25401.csv';
    var csv = "";
    var data = {};

    function buildHierarchy(csv) {
        // Helper function that transforms the given CSV into a hierarchical format.
        const root = { name: "root", children: [] };
        for (let i = 0; i < csv.length; i++) {
            const sequence = csv[i][0];
            const size = +csv[i][1];
            if (isNaN(size)) {
                // e.g. if this is a header row
                continue;
            }
            const parts = sequence.split("-");
            let currentNode = root;
            for (let j = 0; j < parts.length; j++) {
                const children = currentNode["children"];
                const nodeName = parts[j];
                let childNode = null;
                if (j + 1 < parts.length) {
                    // Not yet at the end of the sequence; move down the tree.
                    let foundChild = false;
                    for (let k = 0; k < children.length; k++) {
                        if (children[k]["name"] == nodeName) {
                            childNode = children[k];
                            foundChild = true;
                            break;
                        }
                    }
                    // If we don't already have a child node for this branch, create it.
                    if (!foundChild) {
                        childNode = { name: nodeName, children: [] };
                        children.push(childNode);
                    }
                    currentNode = childNode;
                } else {
                    // Reached the end of the sequence; create a leaf node.
                    childNode = { name: nodeName, value: size };
                    children.push(childNode);
                }
            }
        }
        return root;
    }

    const getData = async function() {
        console.log("calling getData")
        const res = await fetch(url)
        const text = await res.text()
        const csv = d3.csvParseRows(text);
        console.log("number of rows " + csv.length)
        data = buildHierarchy(csv)
        console.log("returning " + data + " children: " + data.children);
        return data;
    }
    return getData;
})


In [ ]:
var width = 932
var height = 932
var breadcrumbWidth = 75
var breadcrumbHeight = 30
var width = 640
var radius = 320

function breadcrumbPoints(d, i) {
  const tipWidth = 10;
  const points = [];
  points.push("0,0");
  points.push(`${breadcrumbWidth},0`);
  points.push(`${breadcrumbWidth + tipWidth},${breadcrumbHeight / 2}`);
  points.push(`${breadcrumbWidth},${breadcrumbHeight}`);
  points.push(`0,${breadcrumbHeight}`);
  if (i > 0) {
    // Leftmost breadcrumb; don't include 6th vertex.
    points.push(`${tipWidth},${breadcrumbHeight / 2}`);
  }
  return points.join(" ");
}

define('sunburst', ['d3'], function(d3) {

    console.log("test1")
  const draw = function(data) {
    console.log("test2")
      console.log(data);
  var partition = data =>
  d3.partition().size([2 * Math.PI, radius * radius])(
    d3
      .hierarchy(data)
      .sum(d => d.value)
      .sort((a, b) => b.value - a.value)
  )

    console.log("test3")

  var color = d3
  .scaleOrdinal()
  .domain(["home", "product", "search", "account", "other", "end"])
  .range(["#5d85cf", "#7c6561", "#da7847", "#6fb971", "#9e70cf", "#bbbbbb"])

  var arc = d3
  .arc()
  .startAngle(d => d.x0)
  .endAngle(d => d.x1)
  .padAngle(1 / radius)
  .padRadius(radius)
  .innerRadius(d => Math.sqrt(d.y0))
  .outerRadius(d => Math.sqrt(d.y1) - 1)

  var mousearc = d3
  .arc()
  .startAngle(d => d.x0)
  .endAngle(d => d.x1)
  .innerRadius(d => Math.sqrt(d.y0))
  .outerRadius(radius)

  const root = partition(data);
  const svg = d3.select("#viz");
  console.log("found svg " + svg)
  // Make this into a view, so that the currently hovered sequence is available to the breadcrumb
  const element = svg.node();
  element.value = { sequence: [], percentage: 0.0 };

  console.log("test4")

  const label = svg
    .append("text")
    .attr("text-anchor", "middle")
    .attr("fill", "#888")
    .style("visibility", "hidden");

  label
    .append("tspan")
    .attr("class", "percentage")
    .attr("x", 0)
    .attr("y", 0)
    .attr("dy", "-0.1em")
    .attr("font-size", "3em")
    .text("");

  label
    .append("tspan")
    .attr("x", 0)
    .attr("y", 0)
    .attr("dy", "1.5em")
    .text("of visits begin with this sequence");

  svg
    .attr("viewBox", `${-radius} ${-radius} ${width} ${width}`)
    .style("max-width", `${width}px`)
    .style("font", "12px sans-serif");

  const path = svg
    .append("g")
    .selectAll("path")
    .data(
      root.descendants().filter(d => {
        // Don't draw the root node, and for efficiency, filter out nodes that would be too small to see
        return d.depth && d.x1 - d.x0 > 0.001;
      })
    )
    .join("path")
    .attr("fill", d => color(d.data.name))
    .attr("d", arc);

  svg
    .append("g")
    .attr("fill", "none")
    .attr("pointer-events", "all")
    .on("mouseleave", () => {
      path.attr("fill-opacity", 1);
      label.style("visibility", "hidden");
      // Update the value of this view
      element.value = { sequence: [], percentage: 0.0 };
      element.dispatchEvent(new CustomEvent("input"));
    })
    .selectAll("path")
    .data(
      root.descendants().filter(d => {
        // Don't draw the root node, and for efficiency, filter out nodes that would be too small to see
        return d.depth && d.x1 - d.x0 > 0.001;
      })
    )
    .join("path")
    .attr("d", mousearc)
    .on("mouseenter", (event, d) => {

        // Get the ancestors of the current segment, minus the root
        const sequence = d
            .ancestors()
            .reverse()
            .slice(1);
            console.log("mouseenter1")

        // Highlight the ancestors
        path.attr("opacity", node =>
            sequence.indexOf(node) >= 0 ? 1.0 : 0.3
        );
        //path.attr("opacity", 0.5)
        console.log("mouseenter2")
        
        const percentage = ((100 * d.value) / root.value).toPrecision(3);

        label
            .style("visibility", null)
            .select(".percentage")
            .text(percentage + "%");

        // Update the value of this view with the currently hovered sequence and percentage
        element.value = { sequence, percentage };
        element.dispatchEvent(new CustomEvent("input"));
            console.log("mouseenter3")
    });

  }
  return draw;
})

test1

In [ ]:
require(['getfiledata', 'sunburst'], function(getData, draw) {

    getData().then(draw)
  
  })
  